# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data=pd.read_csv('../city_info.csv')
city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Baker City,44.7749,-117.8344,38.59,87,33,12.82,US,1666544296
1,Tuatapere,-46.1333,167.6833,48.07,96,100,12.95,NZ,1666544296
2,Saint-Pierre,-21.3393,55.4781,73.08,78,20,2.30,RE,1666544297
3,Arraial do Cabo,-22.9661,-42.0278,76.96,73,40,12.66,BR,1666544297
4,Ushuaia,-54.8000,-68.3000,55.06,47,40,4.61,AR,1666544206


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = city_data[["Lat", "Lng"]].astype(float)
humidity= city_data["Humidity"].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
hotel_df=city_data[(city_data["Max Temp"]<80) & (city_data["Max Temp"]>70) & (city_data["Wind Speed"]<10) & (city_data["Cloudiness"]==0)].dropna()

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,Gerāsh,27.6652,54.1371,73.54,24,0,3.91,IR,1666544316
73,Bāfq,31.6128,55.4107,74.44,16,0,4.43,IR,1666544329
146,Saint-François,46.4154,3.9054,74.64,43,0,9.22,FR,1666544362
160,Rivadavia,-33.1905,-68.4608,77.23,25,0,9.73,AR,1666544368
250,Sabha,27.0377,14.4283,70.90,22,0,9.69,LY,1666544416
402,Najrān,17.4924,44.1277,77.65,22,0,5.91,SA,1666544329
405,San Félix,-32.3333,-58.1333,70.38,41,0,6.49,UY,1666544500
508,Eldorado,-23.7869,-54.2836,79.45,61,0,5.10,BR,1666544548


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df["Hotel Name"]= ''
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    'keyword':"Hotel"
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat= row["Lat"]
    lng= row['Lng']
    params['location']= f'{lat},{lng}'
    hotel= requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index,'Hotel Name']= hotel['results'][0]['name']
    except:
        hotel_df.loc[index,'Hotel Name']= 'N/A'

In [6]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
42,Gerāsh,27.6652,54.1371,73.54,24,0,3.91,IR,1666544316,Chelpasin
73,Bāfq,31.6128,55.4107,74.44,16,0,4.43,IR,1666544329,Hotel
146,Saint-François,46.4154,3.9054,74.64,43,0,9.22,FR,1666544362,Maison Vin d'Hus
160,Rivadavia,-33.1905,-68.4608,77.23,25,0,9.73,AR,1666544368,Hotel Pasambay
250,Sabha,27.0377,14.4283,70.90,22,0,9.69,LY,1666544416,فندق المواجيد
402,Najrān,17.4924,44.1277,77.65,22,0,5.91,SA,1666544329,N/A
405,San Félix,-32.3333,-58.1333,70.38,41,0,6.49,UY,1666544500,Gran Hotel Paysandú
508,Eldorado,-23.7869,-54.2836,79.45,61,0,5.10,BR,1666544548,Villa Verde Hotel


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
42,27.6652,54.1371
73,31.6128,55.4107
146,46.4154,3.9054
160,-33.1905,-68.4608
250,27.0377,14.4283
402,17.4924,44.1277
405,-32.3333,-58.1333
508,-23.7869,-54.2836


In [8]:
# Add marker layer ontop of heat map
marker_locations = locations[['Lat', 'Lng']]

# Create a marker_layer using the poverty list to fill the info box
markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))